In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [2]:
!conda list scipy

# packages in environment at C:\ProgramData\Anaconda3\envs\DataScienceRetreat:
#
# Name                    Version                   Build  Channel
scipy                     1.1.0            py36h4f6bf74_1  


In [52]:
#C:\Users\p.schambach\Desktop\DSR\drone_steering\data\archive\gesture_original\features_flip_c_03_120.csv
data_path = '../../data/archive/gesture_original/features_flip_c_03_120.csv'
df_orig = pd.read_csv(data_path,sep=',')

In [55]:
class DataFrameInterpolator():
    
    def __init__(self):
        self.__can_display = False
    
    def get_new_df(self, df, frmlen):
        
        self.__can_display = False
        self.__get_new_t(df, frmlen)
        self.__get_interpolation_functions(df)
        self.new_df = self.__create_new_df()    
        self.__can_display = True
        
        return self.new_df
    
    
    def __get_new_t(self, df, frmlen):
        
        self.frmlen = frmlen
        self.__orig_shape = df.shape
        self.t = df["ms_since_start"].values
        
        t_min, t_max  = int(self.t[0]), int(self.t[-1])
        diff = t_max - t_min
        
        if diff % frmlen == 0:
            num_frames = diff // frmlen 
        else:
            num_frames = diff // frmlen + 1
            
        self.t_new = np.arange(num_frames+1) * frmlen + t_min
        

        
    def __get_interpolation_functions(self, df):
        
        self.features = list(df.filter(regex = '_(x|y)$', axis = 1).columns)
        self.cubic_interpolation_functions = {}

        for feat in self.features:
            f = df[feat].values
    
            cub_f = interp1d(
                self.t, f, 
                kind = 'cubic', 
                fill_value = (f[0],f[-1]), 
                bounds_error = False,
                assume_sorted=True
            )
        
            self.cubic_interpolation_functions[feat] = cub_f
            
            
     
    def __create_new_df(self):
        new_df = pd.DataFrame(columns=self.features)
        new_df["ms_since_start"] = self.t_new

        for feat in self.features:
            new_df[feat] = self.cubic_interpolation_functions[feat](self.t_new)
            
        return new_df
    
    def display_information(self):
        if self.__can_display:
            print("FrameLength in Milliseconds:", self.frmlen,'\n')
            print("Original Time:")
            print("\tNumber of Frames:", len(self.t))
            print("\t",self.t[0:5],"...",self.t[-5:], '\n')
            print("New Time:")
            print("\tNumber of Frames:", len(self.t_new))
            print("\t",self.t_new[0:5],"...",self.t_new[-5:],'\n')
            print("Original DataFrame Shape:", self.__orig_shape)
            print("New DataFrame Shape:", self.new_df.shape)

In [61]:
di = DataFrameInterpolator()
di.display_information()
new_df = di.get_new_df(df_orig, 37)
di.display_information()

FrameLength in Milliseconds: 37 

Original Time:
	Number of Frames: 515
	 [  0. 121. 245. 368. 487.] ... [63125. 63254. 63377. 63504. 63629.] 

New Time:
	Number of Frames: 1721
	 [  0  37  74 111 148] ... [63492 63529 63566 63603 63640] 

Original DataFrame Shape: (515, 18)
New DataFrame Shape: (1721, 17)
